# Imports

In [7]:
import os
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.config import Config

# Constants

In [18]:
# Change path depending on where data is located
data_root = 'LOCAL_PATH_TO_DATA'
column_names = ['ID', 'DATE', 'ELEMENT', 'DATA VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']

# Data

### Setup Bucket connection

In [3]:
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
s3_resource = boto3.resource("s3", config=Config(signature_version=UNSIGNED))
bucket = s3_resource.Bucket('noaa-ghcn-pds')

### Download files in batches

In [6]:
size = 0
for obj in bucket.objects.all(): 
    if 'by_station' not in obj.key: 
        size += 1 
    else:
        break

print('Number of yearly files: ', size)

Number of yearly files:  260


In [26]:
for obj in bucket.objects.all():
    if 'by_station' not in obj.key:
        _, filename = os.path.split(obj.key)
        local_path = data_root + filename
        if not os.path.isfile(local_path):
            print(filename, ' downloaded')
            s3_client.download_file('noaa-ghcn-pds', obj.key, local_path)
    else:
        break

### Load file

In [27]:
def load_df(year):
    df = pd.read_csv(data_root + str(year) + '.csv.gz', header = 0, names = column_names)
    return df

In [28]:
load_df(1887)

,ID,DATE,ELEMENT,DATA VALUE,M-FLAG,Q-FLAG,S-FLAG,OBS-TIME
0,AGE00135039,18870101,TMIN,50,NaN,NaN,E,NaN
1,AGE00135039,18870101,PRCP,0,NaN,NaN,E,NaN
2,AGE00147705,18870101,TMAX,120,NaN,NaN,E,NaN
3,AGE00147705,18870101,TMIN,36,NaN,NaN,E,NaN
4,AGE00147705,18870101,PRCP,72,NaN,NaN,E,NaN
...,...,...,...,...,...,...,...,...
1013681,USW00094728,18871231,TMIN,-111,NaN,NaN,0,NaN
1013682,USW00094728,18871231,PRCP,48,NaN,NaN,0,NaN
1013683,USW00094728,18871231,SNOW,38,NaN,NaN,Z,NaN
1013684,UZM00038457,18871231,PRCP,0,NaN,NaN,r,NaN
